In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
maxlen=100
!pip install scikit-plot
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
import time
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report,confusion_matrix,balanced_accuracy_score
import scikitplot as skplt
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import doc2vec
from gensim.models import Doc2Vec
from sklearn import utils
from gensim.models.doc2vec import TaggedDocument
from keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from google.colab import files


In [ ]:
def preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
      ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    #text=remov(text)
    ## Tokenize (convert from string to list)
    lst_text = text.split()



    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    text = " ".join(lst_text)
    return text
lst_stopwords=['we','http','#',"@user"  'our',  'ours',  'ourselves',  'you',  "you're",  "you've",  "you'll",  "you'd",  'your',  'yours',  'yourself',  'yourselves',  'he',  'him',  'his',  'himself',  'she',  "she's",  'her',  'hers',  'herself',  'it',  "it's",  'its',  'itself',  'they',  'them',  'their',  'theirs',  'themselves',  'what',  'which',  'who',  'whom',  'this',  'that',  "that'll",  'these',  'those',  'am',  'is',  'are',  'was',  'were',  'be',  'been',  'being',  'have',  'has',  'had',  'having',  'do',  'does',  'did',  'doing',  'a',  'an',  'the',  'and',  'but',  'if',  'or',  'because',  'as',  'until',  'while',  'of',  'at',  'by',  'for',  'with',  'about',  'against',  'between',  'into',  'through',  'during',  'before',  'after',  'above',  'below',  'to',  'from',  'up',  'down',  'in',  'out',  'on',  'off',  'over',  'under',  'again',  'further',  'then',  'once',  'here',  'there',  'when',  'where',  'why',  'how',  'all',  'any',  'both',  'each',  'few',  'more',  'most',  'other',  'some',  'such',  'no',  'nor',  'not',  'only',  'own',  'same',  'so',  'than',  'too',  'very',  's',  't',  'can',  'will',  'just',  'don',  "don't",  'should',  "should've",  'now',  'd',  'll',  'm',  'o',  're',  've',  'y',  'ain',  'aren',  "aren't",  'couldn',  "couldn't",  'didn',  "didn't",  'doesn',  "doesn't",  'hadn',  "hadn't",  'hasn',  "hasn't",  'haven',  "haven't",  'isn',  "isn't",  'ma',  'mightn',  "mightn't",  'mustn',  "mustn't",  'needn',  "needn't",  'shan',  "shan't",  'shouldn',  "shouldn't",  'wasn',  "wasn't",  'weren',  "weren't",  'won',  "won't",  'wouldn',  "wouldn't"]


In [ ]:
def lstm(files,twe,user,loc):

  final_ds="/content/drive/MyDrive/FINAL/twitter-suicidal_data.csv"
  df=pd.read_csv(final_ds,encoding='utf-8',sep=',')
  df=df.dropna()

    #preprocessing
  x_tra= df["tweet"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=lst_stopwords))
  x_tra
  x_final=files[1]
  word_tokenizer = Tokenizer()
  word_tokenizer.fit_on_texts(x_tra)
  x_finals = word_tokenizer.texts_to_sequences(x_final)
  x_finals = pad_sequences(x_finals, padding='post', maxlen=maxlen)

  lstm=tf.keras.models.load_model('/content/drive/MyDrive/FINAL/lstm.h5') #loading mdoel
  #cnn=tf.keras.models.load_model('/content/drive/MyDrive/FINAL/cnn.h5')
  pred = lstm.predict(x_finals, verbose=0).round()  #prediction
  ans=pred.flatten()

    #print(name)
  ad = pd.DataFrame({'Tweet':twe,'User':user,'Location':loc,'SI':ans})
  #rslt_df = ad[ad['SI']==1.0]
  d=ad.groupby(['User','Location'], as_index=False).sum()
  d['Level'] = d['SI'].apply(lambda x: 'STi' if x < 3.0 else('PT' if x<5.0 else 'DT'))
  return d,ad

In [ ]:
from google.colab import files
import io

#files="/content/drive/MyDrive/FINAL/final.csv"

filez = files.upload()
dff=pd.read_csv(io.BytesIO(filez['test.csv']),header=None) #(files,encoding='utf-8',sep=',',header=None)
dff=dff.drop(0,axis=0)
#dff=dff.head(100)
lis=[]
name=[]
loc=[]
lis=dff[1]
name=dff[4]
loc=dff[6]
while True:
  try:
    a,b=lstm(dff,lis,name,loc)
    break
  except (AttributeError,ValueError):
    continue

fin=a[['User','Location','Level']]
leg=['STi - Safe to ignore','PT - Possibly troubling','DT - Deeply Troubling']
print(leg)
fin.sort_values(by=['Level'])


Saving test.csv to test.csv
['STi - Safe to ignore', 'PT - Possibly troubling', 'DT - Deeply Troubling']


<ipython-input-4-dfffbf66515a>:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  d=ad.groupby(['User','Location'], as_index=False).sum()


,User,Location,Level
1,BEN3H0T,camp half-blood,DT
0,1DJjigsaw,Atlanta,PT
7,maybethatswhyy,Admin uses he/it,PT
8,whichxwitch,"glasgow, scotland",PT
2,Erhmrvr,Out of sight,STi
3,ItsSebHdz,"Neo Psychiko, Greece",STi
4,RitersOfRohan,Debt,STi
5,allie__jandro,Allegheny County,STi
6,beequietlex,sav,STi
